fortest内のfilesからdfを作成し(一行ずつappend)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt         
import os

In [29]:
def make_df_from_log_path(log_path): #df_toappendの行となる配列を返す
    #ファイル名からわかる情報 step共通
    file_name = os.path.basename(log_path)
    key_value = file_name.split(':')
    model_title = key_value[0]
    # print(model_title)
    previous_total_asset = int(key_value[1])
    dates = key_value[-1][:21]
    dates = dates.split('~')
    start_date = dates[0]
    end_date = dates[1]
    with open(log_path, 'r') as file:
        lines = file.readlines()
        hoge = []
        #文頭からわかる情報 step共通
        search_strings = ["INPUT_TEXT:", "USING_MODEL:", "args.epsilon:", "HMAX_NORMALIZE:", "SELECT_ACTION:",
                        "TRANSACTION_FEE_PERCENT", "args.bsize:"
        ]
        
        data_1 = []
        data_1.append(log_path)
        # data_1.append(model_title)
        data_1.append(start_date)
        data_1.append(end_date)
        for search_string in search_strings:
            for i, line in enumerate(lines):
                if search_string in line:
                    result = line.replace(search_string, "")
                    result = result.replace(" ", "")
                    result = result.replace("\n", "")
                    # print(result)
                    data_1.append(result)
                    break
                
        #stepごとの結果
        search_strings_2 = ["end_total_asset:", "total_reward:", "total_cost:", "total trades:", ]
        for i, line in enumerate(lines):
            if "test model:" in line: #stepごとの切り分け
                data_2 = []
                data_3 = []
                # print(line)
                step=line.split('_')[-1]
                step = step.replace("\n", "")
                # print(step)
                data_2.append(step)
                data_2.append(previous_total_asset)
                temp_lines=lines[i:]
                for search_string in search_strings_2: #search_strings_2内の処理
                    for j, temp_line in enumerate(temp_lines):
                        if search_string in temp_line:
                            # print(temp_line)
                            result = temp_line.replace(search_string, "")
                            result = result.replace(" ", "")
                            result = result.replace("\n", "")
                            # print(result)
                            data_2.append(result)
                            
                            #profit_ratioの追加
                            if search_string=="end_total_asset:":
                                ratio = float(result)/previous_total_asset
                                # print(ratio)
                                data_2.append(ratio)
                            break
                
                for j, temp_line in enumerate(temp_lines):#[Evaluate]行の処理
                    if "[Evaluate]" in temp_line:
                        # print(temp_line)
                        key_value = temp_line.split(' ')
                        key_value = key_value[2:]
                        # print(key_value)
                        for value in key_value:
                            value = value.split(':')
                            result = value[-1].replace(" ", "")
                            result = result.replace("\n", "")
                            result = float(result)
                            # print(result)
                            data_3.append(result)
                        break
                # break
                data = data_1 + data_2 + data_3
                print(data)
                hoge.append(data)
        return hoge

In [25]:
def append_new_row(log_path, df): #dfにlog_pathの列を追加
    new_row_datas = make_df_from_log_path(log_path)
    for new_row_data in new_row_datas:
        # print(new_row_data)
        new_row_df = pd.DataFrame([new_row_data], columns=df.columns)
        # concat 関数を使用して行を追加
        df = pd.concat([df, new_row_df], ignore_index=True)
    
    return df

In [26]:
columns = ["path", "start_date", "end_date", "INPUT_TEXT", "USING_MODEL", "args.epsilon", "HMAX_NORMALIZE", "SELECT_ACTION",
            "TRANSACTION_FEE_PERCENT", "args.bsize", "step", 
            "previous_total_asset", "end_total_asset", "profit_ratio", "total_reward", "total_cost", "total trades", 
            "mean_reward", "mean_sharpe", "mean_sortino", "mean_calmar", "mean_mdd", "mean_cum_returns"
        ]
df = pd.DataFrame(columns=columns) #初期化
# "model_title",

In [ ]:
log_path = "/home/fukuda/nohup_test_outputs/SEC_only_n1:10000:2015-12-01~2015-12-31.log"
df = append_new_row(log_path, df)
print(df)

In [ ]:
#/home/fukuda/nohup_test_outputsの全ての結果を一つのdfにまとめる

log_folder = "/home/fukuda/nohup_test_outputs"
log_files = os.listdir(log_folder)
for log_file in log_files:
    # print(log_file)
    try:
        log_path = os.path.join(log_folder, log_file)
        df = append_new_row(log_path, df)
    except Exception as e:
        print(f"error:{log_file}:{e}")

    # break

In [28]:
print(df)
# df.to_csv('/home/fukuda/kc_profit_naacl/all_nohup_test_0124.csv', index=False)

                                                  path  start_date  \
0    /home/fukuda/nohup_test_outputs/withoneSEC_tf_...  2015-11-01   
1    /home/fukuda/nohup_test_outputs/withoneSEC_tf_...  2015-11-01   
2    /home/fukuda/nohup_test_outputs/withoneSEC_tf_...  2015-11-01   
3    /home/fukuda/nohup_test_outputs/withoneSEC_tf_...  2015-11-01   
4    /home/fukuda/nohup_test_outputs/withoneSEC_tf_...  2015-11-01   
..                                                 ...         ...   
190  /home/fukuda/nohup_test_outputs/withoneSEC_tf_...  2015-10-01   
191  /home/fukuda/nohup_test_outputs/withoneSEC_tf_...  2015-10-01   
192  /home/fukuda/nohup_test_outputs/withoneSEC_tf_...  2015-10-01   
193  /home/fukuda/nohup_test_outputs/withoneSEC_tf_...  2015-10-01   
194  /home/fukuda/nohup_test_outputs/withoneSEC_tf_...  2015-10-01   

       end_date        INPUT_TEXT USING_MODEL args.epsilon HMAX_NORMALIZE  \
0    2015-11-30           withSEC     model_4        50000             10   
1    